Data consists of four and a half years of wine sales. Some rows are used for keeping track of sample batches, or other sales information. This extra data will be removed and the cleaned data will be further processed for sales trend analysis.

In [48]:
import pandas as pd
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

archive_2019_df = pd.read_excel('2019_WINE_ARCHIVE.xlsx', header = 2)
archive_2020_df = pd.read_excel('2020_WINE_ARCHIVE.xlsx', header = 2)
archive_2021_df = pd.read_excel('2021_WINE_ARCHIVE.xlsx', header = 2)
archive_2022_df = pd.read_excel('2022_WINE_ARCHIVE.xlsx', header = 2)
archive_2023_df = pd.read_excel('2023_WINE_ARCHIVE.xlsx', header = 2)

archives = [archive_2019_df, archive_2020_df, archive_2021_df, archive_2022_df, archive_2023_df]

In [51]:
archive_2019_df.drop(['CUSTOMER', 'TELEPHONE NO.'], axis=1).head()

,BATCH #,DATE,WINE PURCHASED,PRICE,Status,Unnamed: 7,Unnamed: 8
0,NaN,2019-01-03,Water,200,NaN,NaN,NaN
1,NaN,2019-01-03,Kits,1269.79,NaN,NaN,NaN
2,NaN,2019-01-03,Juice,1143.68,NaN,NaN,NaN
3,101,2019-01-03,5 WK Wash. Riesling WV,160,NaN,NaN,NaN
4,102,2019-01-03,8 WK N. Z. SAUV BLANC EC,243,NaN,NaN,NaN


In [52]:
#Want to drop all entries containing samples or the owner's batches
for archive in archives:
  archive.rename({'Unnamed: 7':'NOTES1', 'Unnamed: 8': 'NOTES2'}, axis=1, inplace=True) #Give the last two notes columsn meaningful names
  archive.dropna(how='all', inplace=True) #Drop N/A rows
  archive.dropna(subset=['BATCH #', 'CUSTOMER'], inplace=True)
  archive.fillna('None',inplace=True)

In [53]:
#The kit time information is only included in 2019 and isn't important for our purposes
archive_2019_df[['KIT WEEKS','WINE PURCHASED']] = archive_2019_df['WINE PURCHASED'].str.split('WK', expand = True)
archive_2019_df.drop(['KIT WEEKS'], axis=1, inplace=True)

In [54]:
#Combine into one master archive
archive_df = pd.concat([archive_2019_df, archive_2020_df, archive_2021_df, archive_2022_df])
archive_df['WINE PURCHASED'] = archive_df['WINE PURCHASED'].str.lower()

In [55]:
dropstrings = ['E L C', 'Sample', 'SAMPLE', 'VOID', ' & ERIC']

for substring in dropstrings:
  archive_df = archive_df[~archive_df['CUSTOMER'].str.contains(substring)]

archive_df.to_csv('full_wine_archive.csv')

In [56]:
archive_df.drop(['CUSTOMER', 'TELEPHONE NO.'], axis=1).head()

,BATCH #,DATE,WINE PURCHASED,PRICE,Status,NOTES1,NOTES2
3,101,2019-01-03,wash. riesling wv,160,None,None,None
4,102,2019-01-03,n. z. sauv blanc ec,243,None,None,None
5,103,2019-01-04,white zinfandel vr + res pak,155,None,None,None
6,104,2019-01-04,sauvignon blanc vr,5.2,Gift Cert #588 for $150 used in full 1/04/19,None,None
7,105,2019-01-04,chile corazon le 18,230,LE 18 KIT Paid in full - Back Feb 28th,Pink Pinot done just before,None


In [ ]:
#Use these to find strings that need replacement, removal, or splitting into their own column
wine_vc = archive_df['WINE PURCHASED'].value_counts().sort_values()
wine_vc.to_csv('wine_valcounts.csv')
cust_vc = archive_df['CUSTOMER'].value_counts()
cust_vc.to_csv('cust_valcounts.csv')

In [ ]:
#Clean and Anonymize the Customer Database
archive_df['CUSTOMER'] = archive_df['CUSTOMER'].str.rstrip('123#BCWLNE')
archive_df['CUSTOMER'] = archive_df['CUSTOMER'].str.strip()

customer_str_replacements = {' RIP' : ''
                             , ' BW' : ''
                             , ' CW' : ''
                             , ' WL' : ''
                             , ' NE' : ''}

for key, value in customer_str_replacements.items():
  archive_df['CUSTOMER'] = archive_df['CUSTOMER'].str.replace(key, value)

In [57]:
cust_df = archive_df[['CUSTOMER', 'TELEPHONE NO.']]
cust_df = cust_df.drop_duplicates(subset = ['CUSTOMER', 'TELEPHONE NO.'],keep = 'last')
cust_df['CUST_ID'] = range(1,len(cust_df)+1)
cust_df['CUST_ID'] = cust_df['CUST_ID'].astype(object)

print(np.shape(cust_df))

(1053, 3)


In [58]:
archive_df.dtypes
cust_df.dtypes

BATCH #                   object
DATE              datetime64[ns]
WINE PURCHASED            object
CUSTOMER                  object
TELEPHONE NO.             object
PRICE                     object
Status                    object
NOTES1                    object
NOTES2                    object
dtype: object

CUSTOMER         object
TELEPHONE NO.    object
CUST_ID          object
dtype: object

In [59]:
anon_archive_df = pd.merge(archive_df,cust_df,on=['CUSTOMER','TELEPHONE NO.'],how='left')
anon_archive_df.drop(['CUSTOMER', 'TELEPHONE NO.'], axis=1, inplace=True)
anon_archive_df.head()

,BATCH #,DATE,WINE PURCHASED,PRICE,Status,NOTES1,NOTES2,CUST_ID
0,101,2019-01-03,wash. riesling wv,160,None,None,None,926
1,102,2019-01-03,n. z. sauv blanc ec,243,None,None,None,329
2,103,2019-01-04,white zinfandel vr + res pak,155,None,None,None,751
3,104,2019-01-04,sauvignon blanc vr,5.2,Gift Cert #588 for $150 used in full 1/04/19,None,None,484
4,105,2019-01-04,chile corazon le 18,230,LE 18 KIT Paid in full - Back Feb 28th,Pink Pinot done just before,None,1


In [60]:
anon_archive_df.to_csv('wine_sales_archive.csv')